# Lab: Flowers Multi Classifier
Learn to classify real world images

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elephantscale/cool-ML-demos/blob/main/image/exercise-2-flowers-multi-classifier.ipynb)

### Runtime
~30 minutes

### Note
Here we are dealing with real world images.  Processing them will required a lot of compute power.  
If you have access to, switch to **GPU** run time!

### Running on Google Colab?
Be sure to upload 'image_utils.py'.  Here is how
- Click on the left side bar
- select the 'Files' section
- Click the upload button and select `image_utils.py` file
- If you change your runtime, you may need to upload the file again


### References
- https://www.tensorflow.org/tutorials/images/classification

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
print ('tensorflow version :', tf.__version__)
tf.config.experimental.list_physical_devices()

In [ ]:
## Loading our custom utils files
import os
import sys
from pathlib import Path


# Hack to download image utils when running on Colab ..etc
import os
import urllib.request

file_url = 'https://raw.githubusercontent.com/elephantscale/es-public/master/deep-learning/image_utils.py'
file_location = "image_utils.py"

if not os.path.exists (file_location):
    file_location = os.path.basename(file_location)
    if not os.path.exists(file_location):
        print("Downloading : ", file_url)
        urllib.request.urlretrieve(file_url, file_location)
print('file_location:', file_location)

## TF-GPU Debug
The following block tests if TF is running on GPU.

In [ ]:
## This block is to tweak TF running on GPU
## You may comment this out, if you are not using GPU

## ---- start Memory setting ----
## Ask TF not to allocate all GPU memory at once.. allocate as needed
## Without this the execution will fail with "failed to initialize algorithm" error

from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

## Step 1: Download Data

Here are some stats on flowers dataset

- Size : 230 MB
- Number of files : ~3600
- Has multiple classes:
   - Dandelion
   - Daisy
   - Tulips
   - Sunflowers
   - Roses


```text
./flowers : files= 3671 , size= 230M
./flowers/training : files= 3087 , size= 201M
./flowers/training/dandelion : files= 776 , size= 43M
./flowers/training/daisy : files= 533 , size= 30M
./flowers/training/tulips : files= 677 , size= 49M
./flowers/training/sunflowers : files= 588 , size= 48M
./flowers/training/roses : files= 513 , size= 33M
./flowers/validation : files= 583 , size= 29M
./flowers/validation/dandelion : files= 122 , size= 5.6M
./flowers/validation/daisy : files= 100 , size= 4.3M
./flowers/validation/tulips : files= 122 , size= 5.8M
./flowers/validation/sunflowers : files= 111 , size= 6.4M
./flowers/validation/roses : files= 128 , size= 6.5M

```

In [ ]:
import os, glob

data_location = 'https://elephantscale-public.s3.amazonaws.com/data/images/flowers.zip'

data_location_local = keras.utils.get_file(fname=os.path.basename(data_location),
                                           origin=data_location, extract=True)
print ('local download file: ', data_location_local)

## Peek inside the directory
download_dir = os.path.dirname(data_location_local)
print ("download dir: ", download_dir )

# print a nice tree
! tree -d $download_dir


In [ ]:
## peek inside our data dir
data_dir = os.path.join(os.path.dirname(data_location_local), 'flowers')
print ('here is how files are organized:')
! tree -d $data_dir

#listing = glob.glob(os.path.join(data_dir, "**/*/"), recursive=True)
#for d in listing:
#    print (d)

print ('local data dir: ', data_dir)
train_dir = os.path.join(data_dir, 'training')
validation_dir = os.path.join(data_dir, 'validation')
print ('train dir:', train_dir)
print ('validation dir:', validation_dir)


### See the images and get some stats

In [ ]:
from image_utils import print_training_validation_stats, display_images_from_dir

print_training_validation_stats(train_dir, validation_dir)

display_images_from_dir(train_dir)

## Step 2: Define some constants

In [ ]:
BATCH_SIZE=128
EPOCHS = 10
IMG_HEIGHT = 150
IMG_WIDTH = 150
APP_NAME = 'flowers'

## Step 3: Create Image Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Here we are rescaling images by dividing it by 255
## We are shuffling the images to increase randomness
## Images are reshaped to 150x150


train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data

## TODO-Later
## Add a train_image_generator with image augmentation

train_image_generator = ImageDataGenerator(
                            rescale=1./255,
                            rotation_range=45,
                            width_shift_range=.15,
                            height_shift_range=.15,
                            horizontal_flip=True,
                            zoom_range=0.5
                            )

validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

## Since we only have two classes (cag/dog), we could have used 'binary' mode
##            class_mode='binary'
## But we are doing this as a 'multi-class-classifier' by specifying
##            class_mode='categorical'
## So the same code can work with multiple classes (cat/dog/horse/fox)

train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           # class_mode='binary'
                                                           class_mode='categorical'
                                                          )


val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              # class_mode='binary'
                                                              class_mode='categorical'
                                                             )

## Step 4: Create a Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

NUM_CLASSES = len(train_data_gen.class_indices)

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D((2,2)),
    Dropout(0.2),  
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.2),  
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.2),  
    Flatten(),
    Dense(512, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

print(model.summary())
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

### Compile the model

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# model.compile(#loss='binary_crossentropy',
#               loss = 'categorical_crossentropy',
#               optimizer=RMSprop(lr=0.001),
#               metrics=['accuracy'])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Step 5 - Setup Tensorboard

In [ ]:
## This is fairly boiler plate code

import datetime
import os
import shutil

app_name = APP_NAME


# timestamp  = datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

tb_top_level_dir= '/tmp/tensorboard-logs'

tb_app_dir = os.path.join (tb_top_level_dir, app_name)

tb_logs_dir = os.path.join (tb_app_dir, datetime.datetime.now().strftime("%H-%M-%S"))


print ("Saving TB logs to : " , tb_logs_dir)

#clear out old logs
shutil.rmtree ( tb_app_dir, ignore_errors=True )

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_logs_dir, write_graph=True, 
                                                      write_images=True, histogram_freq=1)

## This will embed Tensorboard right here in jupyter!
# ! killall tensorboard  # kill previously running tensorboards
%load_ext tensorboard
%tensorboard --logdir $tb_logs_dir

## Step 6 : Train

In [ ]:
%%time 

history = model.fit(
    train_data_gen,
    steps_per_epoch= train_data_gen.n // train_data_gen.batch_size,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.n // val_data_gen.batch_size,
    callbacks = [tensorboard_callback]
)

### Save the model for reuse later
As you can see training takes a long time.  
Let's save the resulting model, so we can use it quickly without going through training again.

In [ ]:
import os

model_file = APP_NAME + '-model.h5'
model.save(model_file)

model_size_in_bytes = os.path.getsize(model_file)
print ("model saved as '{}',  size = {:,f} bytes / {:,.1f} KB  / {:,.1f} MB".format(model_file, 
                                    model_size_in_bytes, model_size_in_bytes / 1024, 
                                    model_size_in_bytes / (1024*1024) ))

## Step 7 : See Training History

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

## Step 8 : Evaluate the Model 

### 8.1 - Metrics

In [ ]:
from math import ceil

metric_names = model.metrics_names
print ("model metrics : " , metric_names)

metrics = model.evaluate(val_data_gen, batch_size=val_data_gen.batch_size, steps=ceil(val_data_gen.n // val_data_gen.batch_size) )

for idx, metric in enumerate(metric_names):
    print ("Metric : {} = {:,.3f}".format (metric_names[idx], metrics[idx]))

### 8.2 - Predictions and Confusion Matrix

In [ ]:
import numpy as np
from math import ceil


print ("predicting on {:,} test images".format(val_data_gen.n))
# we need a ceiling for steps
predictions = model.predict(val_data_gen, batch_size=val_data_gen.batch_size, 
                            steps=ceil(val_data_gen.n / val_data_gen.batch_size) )
print( 'predictions.shape: ', predictions.shape)
predictions2 = [ np.argmax(p) for p in predictions]

# ## Ensure all predictions match
assert(len(predictions) == len(predictions2) == len(val_data_gen.classes) )

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

print ('predictions : ' , predictions[:10])
print ('prediction2: ' , predictions2[:10])

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

test_labels = val_data_gen.classes
cm = confusion_matrix(test_labels, predictions2, labels = range(0, NUM_CLASSES))
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print ("class index mapping : ", val_data_gen.class_indices)

plt.figure(figsize = (8,6))

# colormaps : cmap="YlGnBu" , cmap="Greens", cmap="Blues",  cmap="Reds"
sns.heatmap(cm, annot=True, cmap="Reds", fmt='d').plot()

## Step 9: Evaluate the model
What do you think of the model?
- Look at learning curve.  Is there over/under fitting going on?
- How is the accuracy?
- What do you think of confusion matrix?

See the visuzling below.

**TODO: Discuss ways to increase accuracy**


## Step 10: Visually Inspecting Results

In [ ]:
from image_utils import plot_prediction_stats_on_all_classes

val_data_gen.reset() # reset back to batch-1
plot_prediction_stats_on_all_classes(model, val_data_gen)

## Cleanup 
Before running the next exercise, run the following cell to terminate processes and free up resources

In [ ]:
## Kill any child processes (like tensorboard)

import psutil
import os, signal

current_process = psutil.Process()
children = current_process.children(recursive=True)
for child in children:
    print('Killing Child pid  {}'.format(child.pid))
    os.kill(child.pid, signal.SIGKILL)
    
## This will kill actual kernel itself
# os.kill(os.getpid(), signal.SIGKILL)